# Importando los modulos necesarios

In [ ]:
# Instalación de dependencias
!pip install qkeras keras-tuner

In [ ]:
    import h5py
    import numpy as np
    import json
    import tensorflow as tf
    tf.config.run_functions_eagerly(True)
    tf.data.experimental.enable_debug_mode()
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
    from tensorflow.keras.layers import Input, MaxPooling2D, Flatten, Dropout, Add
    from tensorflow.keras.models import Model
    import matplotlib.pyplot as plt
    from qkeras import *
    import keras_tuner as kt

#Cargando el dataset de escalogramas

In [ ]:


from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Tesis/Accelerometer_Dataset/accelerometer_BR_256NS_20Scales_cwt_dataset.h5'


In [ ]:
def load_cwt_dataset(file_path='accelerometer_cwt_dataset.h5'):
    """
    Carga el dataset de escalogramas preprocesados desde un archivo HDF5.

    Args:
        file_path: Ruta al archivo HDF5

    Returns:
        dataset: Un diccionario con los datos listos para entrenar
    """
    with h5py.File(file_path, 'r') as hf:
        # Cargar escalogramas y etiquetas (ya normalizados, con canal y one-hot)
        x_train = np.array(hf['train']['scalograms'])
        y_train = np.array(hf['train']['labels_onehot'])  # Usar etiquetas one-hot
        y_train_raw = np.array(hf['train']['labels'])     # También cargar etiquetas sin one-hot

        x_test = np.array(hf['test']['scalograms'])
        y_test = np.array(hf['test']['labels_onehot'])    # Usar etiquetas one-hot
        y_test_raw = np.array(hf['test']['labels'])       # También cargar etiquetas sin one-hot

        # Cargar metadatos
        num_classes = hf['metadata']['num_classes'][()]
        shape = tuple(hf['metadata']['shape'][()])
        max_val = hf['metadata']['max_val'][()]  # Valor máximo usado en normalización

        # Cargar diccionario de etiquetas
        label_codes_dict = json.loads(hf['metadata'].attrs['label_codes_dict'])

    # Organizar datos como un dataset tipo Keras (ya preprocesados)
    dataset = {
        'train': (x_train, y_train, y_train_raw),
        'test': (x_test, y_test, y_test_raw),
        'num_classes': num_classes,
        'input_shape': shape,
        'label_codes_dict': label_codes_dict,
        'max_val': max_val
    }

    print(f"Dataset cargado: {x_train.shape[0]} muestras de entrenamiento, {x_test.shape[0]} muestras de prueba")
    print(f"Forma de cada escalograma: {shape}")
    print(f"Número de clases: {num_classes}")
    print(f"Rango de valores: [0, {max_val}] (ya normalizados)")

    # Verificar que ya están normalizados
    print(f"Valor máximo en datos de entrenamiento: {np.max(x_train)}")
    print(f"Valor mínimo en datos de entrenamiento: {np.min(x_train)}")

    return dataset

In [ ]:
# Carga el dataset (ya preprocesado)
file_path = '/content/drive/MyDrive/Tesis/Accelerometer_Dataset/accelerometer_cwt_dataset_ns512_processed.h5'
dataset = load_cwt_dataset(file_path)

# Obtén las partes del dataset (ya normalizadas y con one-hot)
x_train, y_train, y_train_raw = dataset['train']
x_test, y_test, y_test_raw = dataset['test']
input_shape = dataset['input_shape']
num_classes = dataset['num_classes']

print("Datos listos para entrenamiento:")
print(f"x_train: {x_train.shape}, y_train: {y_train.shape}")
print(f"x_test: {x_test.shape}, y_test: {y_test.shape}")

#Verificando la normalización de los datos

In [ ]:
# Verificar si los datos ya están normalizados
print("Estado actual de los datos:")
print("Valor máximo en x_train:", np.max(x_train))
print("Valor mínimo en x_train:", np.min(x_train))
print("Media de x_train:", np.mean(x_train))
print("Desviación estándar de x_train:", np.std(x_train))

# Visualizar la distribución de valores
plt.figure(figsize=(10, 4))
plt.hist(x_train.flatten(), bins=50)
plt.title('Distribución de valores en x_train')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

#Separamos una parte de los datos para la validacion del modelo

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos
x_train_final, x_val, y_train_final, y_val = train_test_split(
    x_train, y_train,
    test_size=0.2,  # 20% para validación
    random_state=42,  # Para reproducibilidad
    stratify=y_train  # Mantener la distribución de clases
)

# Verificar tamaños
print(f"Datos de entrenamiento: {x_train_final.shape[0]} muestras")
print(f"Datos de validación: {x_val.shape[0]} muestras")

# Modelo CNN básico (Tiny)

In [ ]:
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Crear un modelo CNN pequeño con técnicas modernas
def create_tiny_cnn_model(input_shape, num_classes):
    """
    Crea un modelo CNN compacto optimizado para microcontroladores.
    """
    model = keras.Sequential([
        # Capa de entrada
        layers.Input(shape=input_shape),
        
        # Bloque 1: 8 filtros, escalado gradual
        layers.Conv2D(8, kernel_size=(3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        # Bloque 2: 16 filtros con conexión residual
        layers.Conv2D(16, kernel_size=(3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        # Bloque 3: 32 filtros (más capas en etapas posteriores)
        layers.Conv2D(32, kernel_size=(3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        # Capa de salida
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),  # Dropout para reducir overfitting
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model
# Crear el modelo
input_shape = x_train_final.shape[1:]
num_classes = 5
tiny_cnn = create_tiny_cnn_model(input_shape, num_classes)
tiny_cnn.summary()

## Entrenamiento del modelo

In [ ]:
# Compilar modelo
tiny_cnn.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks para mejorar el entrenamiento
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    ModelCheckpoint('best_tiny_cnn_model.h5', save_best_only=True, monitor='val_accuracy')
]

# Entrenar el modelo
history = tiny_cnn.fit(
    x_train_final, y_train_final,
    batch_size=64,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks
)

# Evaluacion del modelo

In [ ]:
# Visualizar el proceso de entrenamiento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Loss')
plt.legend()
plt.show()

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = tiny_cnn.evaluate(x_test, y_test)
print(f"Precisión en el conjunto de prueba: {test_acc:.4f}")

# Matriz de confusión
# Matriz de confusión
y_pred = np.argmax(tiny_cnn.predict(x_test), axis=1)
y_true = y_test_raw  # Usar las etiquetas originales sin one-hot
conf_mat = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.ylabel('Etiqueta Real')
plt.xlabel('Etiqueta Predicha')
plt.show()

# Para mostrar nombres en lugar de códigos en reportes
class_names = list(dataset['label_codes_dict'].keys())
print(classification_report(y_test_raw, y_pred, target_names=class_names))

# Modelo Avanzado con Conexiones Residuales

In [ ]:
def create_tiny_resnet_model(input_shape, num_classes):
    """
    Crea un modelo CNN con conexiones residuales.
    """
    inputs = keras.Input(shape=input_shape)
    
    # Bloque 1
    x = layers.Conv2D(8, kernel_size=(3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    # Bloque 2 con conexión residual
    residual = x
    x = layers.Conv2D(16, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # Proyección de la conexión residual
    residual = layers.Conv2D(16, kernel_size=(1, 1), padding='same')(residual)
    x = layers.add([x, residual])  # Conexión residual
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    # Bloque 3 con conexión residual
    residual = x
    x = layers.Conv2D(32, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(32, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    # Proyección de la conexión residual
    residual = layers.Conv2D(32, kernel_size=(1, 1), padding='same')(residual)
    x = layers.add([x, residual])  # Conexión residual
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    # Capa de salida
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    return model

# Crear el modelo mejorado
resnet_tiny = create_tiny_resnet_model(input_shape, num_classes)
resnet_tiny.summary()

# Compilar el modelo
resnet_tiny.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss= 'categorical_crossentropy',
    metrics=['accuracy']
)

# Entrenar el modelo
history_resnet = resnet_tiny.fit(
    x_train_final, y_train_final,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=callbacks
)

# Evaluar en el conjunto de prueba
test_loss, test_acc = resnet_tiny.evaluate(x_test, y_test)
print(f"Precisión en el conjunto de prueba (Modelo ResNet): {test_acc:.4f}")

# Exportación del modelo para STM32CubeAI

In [ ]:
# Convertir a TFLite (formato intermedio para STM32CubeAI)
def convert_to_tflite(model, model_name="tiny_cnn_model"):
    # Convertir el modelo a TFLite
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    
    # Guardar el modelo TFLite
    tflite_model_path = f"{model_name}.tflite"
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    
    print(f"Modelo guardado como {tflite_model_path}")
    
    # También guardar el modelo original en formato .h5
    model.save(f"{model_name}.h5")
    print(f"Modelo original guardado como {model_name}.h5")
    
    return tflite_model_path

# Convertir ambos modelos
tflite_basic = convert_to_tflite(tiny_cnn, "tiny_cnn_model")
tflite_resnet = convert_to_tflite(resnet_tiny, "tiny_resnet_model")

print("\nPara utilizar estos modelos con STM32CubeAI:")
print("1. Abra STM32CubeAI")
print("2. Importe el archivo .h5 o .tflite")
print("3. Configure la validación y generación de código C")
print("4. Integre el código generado en su proyecto STM32")

# Neural Architecture Search (NAS) multiobjetivo

In [ ]:
import keras_tuner as kt
from tensorflow.keras import backend as K

# Función de evaluación multiobjetivo personalizada
def multi_obj_score(val_accuracy, model_params):
    # Balancear precisión vs tamaño del modelo
    # Alpha controla la importancia relativa de precisión vs tamaño
    alpha = 0.7  # 70% importancia en precisión, 30% en tamaño
    
    # Normalizar número de parámetros (asumiendo un máximo de 1M params)
    normalized_params = model_params / 1_000_000
    
    # Penalizar modelos muy grandes
    size_score = 1.0 - normalized_params
    
    # Puntuación total (combinación ponderada)
    total_score = alpha * val_accuracy + (1 - alpha) * size_score
    
    return total_score

# Función para construir el modelo con búsqueda de hiperparámetros
def model_builder(hp):
    # Hiperparámetros para buscar
    filters_1 = hp.Int('filters_1', min_value=8, max_value=32, step=8)
    filters_2 = hp.Int('filters_2', min_value=16, max_value=64, step=16)
    filters_3 = hp.Int('filters_3', min_value=32, max_value=128, step=32)
    
    kernel_size = hp.Choice('kernel_size', values=[3, 5])
    
    use_batch_norm = hp.Boolean('batch_norm')
    use_dropout = hp.Boolean('dropout')
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    
    use_residual = hp.Boolean('residual')
    
    # Construir modelo
    inputs = keras.Input(shape=input_shape)
    
    # Primera etapa
    x = layers.Conv2D(filters_1, kernel_size=kernel_size, padding='same')(inputs)
    if use_batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    # Segunda etapa (posiblemente con conexión residual)
    prev_x = x
    
    x = layers.Conv2D(filters_2, kernel_size=kernel_size, padding='same')(x)
    if use_batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    if use_residual:
        # Proyectar residual si es necesario
        if filters_1 != filters_2:
            prev_x = layers.Conv2D(filters_2, kernel_size=1, padding='same')(prev_x)
        x = layers.add([x, prev_x])
    
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    # Tercera etapa
    prev_x = x
    
    x = layers.Conv2D(filters_3, kernel_size=kernel_size, padding='same')(x)
    if use_batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    if use_residual:
        # Proyectar residual si es necesario
        if filters_2 != filters_3:
            prev_x = layers.Conv2D(filters_3, kernel_size=1, padding='same')(prev_x)
        x = layers.add([x, prev_x])
    
    x = layers.GlobalAveragePooling2D()(x)
    
    if use_dropout:
        x = layers.Dropout(dropout_rate)(x)
        
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    
    # Compilar modelo
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Clase personalizada para optimización multiobjetivo
class MultiObjectiveHyperModel(kt.HyperModel):
    def __init__(self):
        super().__init__()
        
    def build(self, hp):
        return model_builder(hp)
        
    def fit(self, hp, model, x_train, y_train, **kwargs):
        return model.fit(
            x_train,
            y_train,
            batch_size=hp.Choice('batch_size', [16, 32, 64]),
            **kwargs
        )

# Clase personalizada para Tuner con evaluación multiobjetivo
class MultiObjectiveTuner(kt.Tuner):
    def on_trial_end(self, trial):
        # Obtener precisión de validación
        val_accuracy = trial.metrics.get_last_value('val_accuracy') or 0
        
        # Obtener número de parámetros del modelo
        model_params = trial.trial.executions[0].model.count_params()
        
        # Calcular puntuación multiobjetivo
        score = multi_obj_score(val_accuracy, model_params)
        
        # Registrar métricas adicionales
        trial.metrics.update({'model_size': model_params, 'multi_obj_score': score})
        
        # Ordenar por la puntuación multiobjetivo en lugar de solo precisión
        self.oracle.objective.name = 'multi_obj_score'
        trial.score = score
        
        super().on_trial_end(trial)

# Inicializar el tuner
tuner = MultiObjectiveTuner(
    hypermodel=MultiObjectiveHyperModel(),
    oracle=kt.oracles.BayesianOptimization(
        objective=kt.Objective('multi_obj_score', direction='max'),
        max_trials=20
    ),
    directory='nas_search',
    project_name='multi_obj_cnn'
)

# Buscar la mejor arquitectura
tuner.search(
    x_train_final, y_train_final,
    validation_data=(x_val, y_val),
    epochs=15,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=5)
    ]
)

# Obtener los mejores modelos y mostrar resultados
best_models = tuner.get_best_models(num_models=3)
best_hps = tuner.get_best_hyperparameters(num_trials=3)

for i, (model, hp) in enumerate(zip(best_models, best_hps)):
    print(f"\nModelo #{i+1}:")
    print(f"Hiperparámetros: {hp.values}")
    model.summary()
    
    # Evaluar en el conjunto de prueba
    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"Precisión en prueba: {test_acc:.4f}")
    print(f"Número de parámetros: {model.count_params():,}")
    
    # Guardar el mejor modelo para STM32CubeAI
    if i == 0:
        convert_to_tflite(model, f"nas_optimized_model")

# Análisis del Modelo Optimizado y Preparación para STM32

In [ ]:
# Obtener el mejor modelo de la búsqueda
best_model = tuner.get_best_models(num_models=1)[0]

# Visualizar la arquitectura
tf.keras.utils.plot_model(best_model, show_shapes=True, dpi=70)

# Análisis del tamaño del modelo
print("\nAnálisis del modelo optimizado:")
print(f"Número total de parámetros: {best_model.count_params():,}")

# Tamaño del modelo en memoria
def get_model_memory_usage(model):
    shapes = [tf.TensorShape(layer.output_shape) for layer in model.layers]
    dtypes = [layer.dtype for layer in model.layers]
    
    memory = 0
    for shape, dtype in zip(shapes, dtypes):
        if shape.is_fully_defined():
            memory += np.prod(shape.as_list()) * np.dtype(dtype.name).itemsize
    
    return memory / (1024 * 1024)  # MB

print(f"Memoria estimada: {get_model_memory_usage(best_model):.2f} MB")

# Cuantización para reducir aún más el tamaño
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Datos representativos para la cuantización
def representative_dataset():
    for i in range(min(100, len(X_train))):
        yield [np.expand_dims(X_train[i], axis=0).astype(np.float32)]

converter.representative_dataset = representative_dataset

# Convertir y guardar modelo cuantizado
tflite_quant_model = converter.convert()
with open('tiny_cnn_quantized.tflite', 'wb') as f:
    f.write(tflite_quant_model)

print("\nModelo cuantizado guardado como 'tiny_cnn_quantized.tflite'")
print("Este modelo está optimizado para su despliegue en STM32 a través de STM32CubeAI")